In [1]:
import skimage.feature, skimage.io
import numpy as np
import math
from math import floor
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from skimage.feature import hog


warnings.filterwarnings("ignore")
#Pasta que contém o conjunto de dados já extraído.
#Esta pasta contém as pastas A-Z e os arquivos NIST_Train_Upper.txt, NIST_Test_Upper.txt e NIST_Valid_Upper.txt
NCharacter_dataset_folder = "./exercicios/NCharacter_SD19_BMP/"

#Uma pasta onde as características extraídas são salvas.
features_folder = 'features'



#Classe que abstrai a divisão em zonas
class Zonas(object):
    def __init__(self):
        pass
    
    #Função geradora que retorna uma zona de cada vez.
    #Os argumentos são o número de linhas da imagem e o número de colunas da imagem.
    def get_zonas(self, n_linhas, n_colunas):
        raise NotImplementedError

#Classe que implementa a divisão em zonas retangulares.
class ZonasRetangulares(Zonas):
    
    #Construtor que configura o zoneamento a ser feito: zonas_x zonas horizontais e zonas_y zonas verticais.
    def __init__(self, zonas_x, zonas_y):
        super(ZonasRetangulares, self).__init__()
        self.zonas_x = zonas_x
        self.zonas_y = zonas_y
        
    #Implementa a função geradora de zonas retangulares.
    def get_zonas(self, n_linhas, n_colunas):
        cortes_x = np.floor(np.linspace(0, n_colunas, num=self.zonas_x+1)).astype(int)
        cortes_y = np.floor(np.linspace(0, n_linhas, num=self.zonas_y+1)).astype(int)
        #print (cortes_x)
        #print (cortes_y)
        for i in range(len(cortes_x)-1):
            for j in range(len(cortes_y)-1):
                yield(cortes_x[i], cortes_y[j], cortes_x[i+1], cortes_y[j+1],cortes_x[i] - cortes_x[i+1], cortes_y[i] - cortes_y[i+1] )


NameError: name 'warnings' is not defined

# Caraterísticas

In [ ]:
def histograma_cor(imagem):
    #print('im', imagem.shape)
    vals, counts = np.unique(imagem, return_counts=True)
    o = vals.argsort()
    vals = vals[o]
    counts = counts[o]
    if len(vals) < 2:
        #imagem com apenas branco ou apenas preto
        if vals[0] == 0:
            return [counts[0], 0]
        else:
            return [0, counts[0]] 
    return counts

def histograma_horizontal(imagem):
    cortes = np.linspace(0, imagem.shape[0], 4)
    j = imagem.shape[1]
    hh = []
    for m in range(0, len(cortes)-1): 
        qtdPreto = 0
        qtdBranco = 0
        for k in range(floor(cortes[m]), floor(cortes[m+1])):            
            for l in range(0, j):
                if imagem[k][l] == 0:
                    qtdPreto += 1
                else:
                    qtdBranco += 1
        concatena = str(qtdBranco) + str(qtdPreto)
        hh.append(int(concatena))
    return hh
        

def histograma_vertical(imagem):
    cortes = np.linspace(0, imagem.shape[1], 4)
    j = imagem.shape[0]
    hv = []
    for m in range(0, len(cortes)-1): 
        qtdPreto = 0
        qtdBranco = 0
        for k in range(floor(cortes[m]), floor(cortes[m+1])):            
            for l in range(0, j):
                if imagem[l][k] == 0:
                    qtdPreto += 1
                else:
                    qtdBranco += 1
        concatena = str(qtdBranco) + str(qtdPreto)
        hv.append(int(concatena))
    return hv

def hog_image(image):
    return hog(image, orientations=4, pixels_per_cell=(1, 1), cells_per_block=(1, 1))



# Extração de Características

In [ ]:
#Esta função apenas abre os arquivos com as listas de treino / teste e validação.
#Retorna os caminhos para os arquivos, os rótulos e o nome dos arquivos sem o caminho.
def parse_filelist(path, prefix=''):
    with open(path, 'r') as f:
        c = f.readlines()
    caminhos = list(map(str.strip, c))
    rotulos = [ i.split('/')[1].upper() for i in caminhos]
    arquivos = [i.split('/')[-1] for i in caminhos]
    p = zip(rotulos, arquivos)
    caminhos = [ prefix + '/' + i[0] + '/' + i[1] for i in p]
    
    return list(zip(caminhos,rotulos, arquivos))

#Esta é uma função que recebe o tipo de zoneamento a ser feito e as features que devem ser
#extraídas de cada zona. Veja que essa função é chamada no "for" abaixo, que faz a extração das características
#para cada uma das listas de imagens (treino, teste e validação)
def extract_features(filelist, dataset_folder, zonas, features=[histograma_cor, histograma_horizontal]):
    instancias = parse_filelist(filelist, prefix=dataset_folder)
    #Note que o MAP abaixo mapeia cada instância (imagem) à função que extrai as 
    #características (feature_extraction) abaixo.
    features = list(map(feature_extraction, instancias, [zonas] * len(instancias), [features] * len(instancias)))
    
    return np.array(features)

#Essa função de extração das características de cada instância. zonas é uma instância de subclasse
#de Zonas. Features é uma lista de funções que extraem características. Cada função da lista recebe uma matriz
#que representa a imagem (que está na zona) e retorna o vetor de característica computado daquela característica. 
def feature_extraction(instancia, zonas, features):
    caminho = instancia[0]
    #print(instancia)
    imagem = skimage.io.imread(caminho)
    caracteristicas = np.array([])
    #print("imagem.shape",imagem.shape)
    
    res = []
    for f in features:
        for z in zonas.get_zonas(imagem.shape[1], imagem.shape[0]):
            #print ("%d:%d,%d:%d" % (z[0], z[2], z[1], z[3]))
            f_val = f(imagem[z[0]:z[2],z[1]:z[3]])
            res.extend(f_val)
    
    return np.array(res)

#Realiza a extração das características!
for i in [('NIST_Train_Upper.txt', 'train'), ('NIST_Test_Upper.txt', 'test'), ('NIST_Valid_Upper.txt', 'val')]:
    print('extraindo características de %s' % (i[0]))
    #note que esta linha extrai características de 4 zonas (2 imagens por linhas e 2 por coluna). 
    #Neste exemplo apenas a característica histograma_cor é computada.
    feats = extract_features(NCharacter_dataset_folder + i[0], NCharacter_dataset_folder, ZonasRetangulares(2,2), features=[histograma_cor, histograma_horizontal])
    
    #Extraia aqui outras características! Escolha outros zoneamentos! (Só não esqueça de concatenar tudo em feats)
    #Estude a função np.concatenate do numpy para concatenar!
    
    #np.save(open(features_folder + ('/%s_feats.pkl' % (i[1])), 'wb'), feats )
    np.save(features_folder + ('/%s_feats.pkl' % (i[1])), feats)

print('Fim da extração de características!')



# Treino e Teste com SVM e KNN (Sem validação)

In [ ]:
train_features = np.load(features_folder + '/train_feats.pkl.npy')
train_rotulos = parse_filelist(NCharacter_dataset_folder + 'NIST_Train_Upper.txt')
train_rotulos = [i[1] for i in train_rotulos]
#print (len(train_rotulos), train_features.shape)

test_features = np.load(features_folder + '/test_feats.pkl.npy')
test_rotulos = parse_filelist(NCharacter_dataset_folder + 'NIST_Test_Upper.txt')
test_rotulos = [i[1] for i in test_rotulos]
print (len(test_rotulos), test_features.shape)

SS = StandardScaler()
SS.fit(train_features)
train_features = SS.transform(train_features)
test_features = SS.transform(test_features)

#KKKKKKKKKKKKKNNNNNNNNNNNNNNNNNNNNNNNNNN
KNN = KNNC(n_neighbors=3)

KNN.fit(train_features, train_rotulos)


y_pred = KNN.predict(test_features)

print('KNN score: ' , '{:.3f}'.format(accuracy_score(test_rotulos, y_pred)))
#print(confusion_matrix(test_rotulos, y_pred))

print(classification_report(test_rotulos, y_pred, digits=3))

#SSSSSSSSSSSVVVVVVVVVVVMMMMMMMMMMMMMMMM
clf = SVC()
clf.fit(train_features, train_rotulos)
y_pred = clf.predict(test_features)

print('SVM score: ', '{:.3f}'.format(accuracy_score(test_rotulos, y_pred)))
#print(confusion_matrix(test_rotulos, y_pred))

print(classification_report(test_rotulos, y_pred, digits=3))


#ARVORE DECISÃO
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_features, train_rotulos)
clf.predict(test_features)
clf.predict_proba(test_features)
